### Chapter 8 函数和闭包

#### 8.1 方法

In [17]:
import scala.io.Source

object LongLines {
    def processFile(filename: String, width: Int) {
        // 文件名用来创建Source对象
        val source = Source.fromFile(filename);
        // 读取指定文件
        for (line <- source.getLines) {
            processLine(filename, width, line);
        }
    }
    
    private def processLine(filename: String, width:Int, line:String) {
        // 打印超过长度设定范围的文本行
        if (line.length > width) {
            println(filename + ": " + line.trim);
        }
    }
}

import scala.io.Source
defined object LongLines

In [28]:
val f = "D:\\work\\test.txt";
val w = 5;
LongLines.processFile(f, w);

D:\work\test.txt: length over 5

f: String = """
D:\work\test.txt
"""
w: Int = 5

In [29]:
// 创建一个应用，把第一个命令行参数当作行长度，并把后续的参数解释为文件名
// 在命令行中使用LongLines

object FindLongLines {
    def main(args: Array[String]) {
        val width = args(0).toInt;
        for (arg <- args.drop(1)) {
            LongLines.processFile(arg, width)
        }
    }
}

defined object FindLongLines

#### 8.2 本地函数

帮助函数的名称可能污染程序的命名空间，一旦函数被打包成可复用的类和对象，最好让类的使用者不可见，因为它们经常不能独立表达什么意思，如果之后用其他方式重写类的话，也常会希望能确保足以删掉这些帮助方法的灵活度

重构8.1中原本的Longlines版本，去掉private修饰符，然后把processLine的定义放在processFile的定义中，此时，processLine的范围局限在processFile之内，外部无法访问

同时，processLine中的filename和width参数，与processFile中比起来并没有发生改变，所以可以直接使用外部processLine函数的参数

In [ ]:
import scala.io.Source

object LongLines {
    def processFile(filename: String, width: Int) {
        // 把函数定义在别的函数之类，就像本地变量一样
        // 这种本地函数仅在包含它的代码块中可见
        def processLine(line:String) {
            // 打印超过长度设定范围的文本行
            if (line.length > width) {
                println(filename + ": " + line.trim);
            }
        }
        
        // 文件名用来创建Source对象
        val source = Source.fromFile(filename);
        // 读取指定文件
        for (line <- source.getLines) {
            processLine(filename, width, line);
        }
    }
    
}

#### 8.3 头等函数

Scala的函数是头等函数(first-class function)，你不仅可以定义和调用函数，还可以把他们携程匿名的字面量，并把它们作为值传递

函数字面量被编译进类，并在运行期实例化为函数值

In [33]:
// 这是对数字执行递增操作的函数字面量的简单例子
// =>指明这个函数把左边的东西转变成右边的东西
(x: Int) => x + 1;

res29: Int => Int = <function1>

函数值是对象，可以将其存入变量，它们也是函数，所以也可以使用通常的括号函数调用写法调用它们

In [34]:
var increase = (x: Int) => x + 1;
increase(0);

increase: Int => Int = <function1>
res30_1: Int = 1

如果想让函数字面量包含多条语句，可以用花括号包住函数体，一行放一条语句，这样就组成了代码块

当函数值被调用时，所有语句将被执行，函数的返回值就是最后一行表达式所产生的值

In [37]:
var increase = (x: Int) => {
    println("the number is ");
    println(x + 1);
    x + 1;
}
increase(0)

the number is 1

increase: Int => Int = <function1>
res33_1: Int = 1

再举一个例子，所有的集合类都可以使用foreach方法

In [38]:
val someNumbers = List(-1, 0, 1);
someNumbers.foreach((x: Int) => println(x));

-1
0
1


someNumbers: List[Int] = List(-1, 0, 1)

关于集合类型的filter方法

In [41]:
// 函数(x:Int)=>x>0可以用来作过滤
someNumbers.filter((x: Int) => x > 0);

res37: List[Int] = List(1)

#### 8.4 函数字面量的短格式

Scala提供了许多方法去除冗余信息，并把函数字面量写得更简短

一种让函数字面量简短的方法是去除参数类型，如下，Scala编译器知道x的类型，因为它看到你立刻使用了这个函数过滤列表(someNumbers暗示），这被称为目标类型化：target typing

还有一种是神略参数外的括号，因为某些参数的类型是被推断的

In [46]:
val someNumbers = List(-1.0, 0.0, 1.0);
someNumbers.filter(x => x > 0);

someNumbers: List[Double] = List(-1.0, 0.0, 1.0)
res42_1: List[Double] = List(1.0)

#### 8.5 占位符语法

如果每个参数在函数字面量内仅出现一次的话，把下划线当作一个或更多参数的占位符，可以让函数字面量更简洁

In [45]:
someNumbers.filter(_ > 0);

res41: List[Double] = List(1.0)

有时把下划线当作参数的占位符，编译器可能无法推断确实的参数类型，此时可以使用冒号指定类型，比如

In [49]:
val f = (_: Int) + (_: Int);
// 第一个下划线代表第一个参数，第二个下划线代表第二个......如此类推
f(5, 10);

f: (Int, Int) => Int = <function2>
res44_1: Int = 15

#### 8.6 部分应用函数

In [50]:
// 函数与下划线之间记得留一个空格
someNumbers.foreach(println _);

-1.0
0.0
1.0


In [57]:
// 也可以写作以下 (这是一个省略所有参数的偏程序表达式)
someNumbers.foreach(println);

-1.0
0.0
1.0


在Scala中，调用函数、传入任何需要的参数，实际上是在把函数应用到参数上

In [52]:
def sum(a: Int, b: Int, c:Int) = a + b + c;
// 将函数sum应用到参数1，2，3上
sum(1, 2, 3);

defined function sum
res47_1: Int = 6

部分应用函数是一种表达式，不需要提供函数需要的所有参数，代之以仅提供部分，或不提供所需参数，如下

In [53]:
val a = sum _;

a: (Int, Int, Int) => Int = <function3>

上面的代码实例化了一个带3个缺失整数参数的函数值，并把这个新的函数值的索引赋给变量a，实际过程是：名为a的变量指向一个函数值对象，这个函数值是由Scala编译器依照部分应用函数表达式sum _自动产生的类的一个实例

In [54]:
a(1, 2, 3);

res49: Int = 6

编译器产生的类有一个apply方法，可以用来带3个参数，编译器把表达式a(1,2,3)翻译成对函数值apply方法的调用，传入3个参数1，2，3

In [55]:
// a(1,2,3)是以下代码的短格式
a.apply(1,2,3);

res50: Int = 6

#### 8.7 闭包

闭包是依照函数字面量在运行是创建的函数值（对象）

如(x: Int) => x + 1 被称为封闭项，但(x: Int) => x + more这种带有自由变量的函数字面量都是开放项，以它为模板在运行期间创建的函数值将必须捕获自由变量more，因此得到的函数值将包含指向捕获的more变量的索引

In [63]:
var n1 = 0.0;
val someNumbers = List(-1.0, 0.0, 2.0);

// Scala的闭包捕获变量本身，而不是变量指向的值
// 所以闭包对捕获变量作出的改变在闭包之外也可见
someNumbers.foreach(n1 += _);

n1: Double = 1.0
someNumbers: List[Double] = List(-1.0, 0.0, 2.0)

#### 8.8 重复参数

Scala中可以指明函数的最后一个参数是重复的，从而允许用户向函数传入可变长度参数列表

想要标注一个重复参数，可在参数的类型后放一个*

In [67]:
def echo(args: String*) = {
    for (arg <- args) {
        println(arg);
    }
}

echo("hello", "world")

hello
world


defined function echo

函数内部，重复参数的类型是声明参数类型的数组，因此，echo函数里被声明为类型String*的args的类型实际上是Array[String]